---
title: "My Document"
jupyter: 
  kernelspec:
    name: "mosaiks"
    language: "python"
    display_name: "MOSAIKS"
---





# Preparing labels {#sec-labels-data-prep}





```{r}
#| echo: false
#| results: "asis"

source("../_common.R")
status("draft")
```





## Overview

In this chapter, we discuss the process of preparing labels for use with MOSAIKS features. Labels are the observed values we aim to predict with our model—such as crop yields, forest cover, or any variable of interest. They can come in many spatial formats (e.g., points, polygons, or gridded rasters), but they must include a spatial component. We use this spatial information to join the labels with MOSAIKS features, which are also spatially explicit.

Although MOSAIKS offers many optional steps, two components are essential:

1. Ground observations (labels)  
2. Satellite features  

Both datasets must align in spatial resolution and contain appropriate geographic data for merging.

## MOSAIKS Grid

Before we talk about labels, we must first understand the resolution that the MOSAIKS features are offered in. The resolution you choose will serve as the target resolution for your summarizing your labels.

### Resolution

The standard resolution of MOSAIKS is a global grid at 0.01° resolution. Each grid cell is approximately 1 km² at the equator. The grid is often represented as a point grid, where each point is the center of a grid cell. This means that standard MOSAIKS features come with a latitude and longitude coordinate, which is the center of the grid cell.

::: {.callout-note}
MOSAIKS grid cell centroids are rounded to 0.005 degrees and are spaced by 0.01 degree (e.g., 10.005, 10.015, 10.025,...).
:::

![Visual representation of a standardized grids at varying resolutions (δ) with the highest resolution on the left, and lower resolutions moving right. Source: [Rolf et al. 2021 Figure 3 c](https://www.nature.com/articles/s41467-021-24638-z/figures/3).](../images/rolf_et_al_2021-Fig_3-c-grid.svg){#fig-grid}

### Advantages of the MOSAIKS grid

The MOSAIKS grid has several advantages. The primary advantage is that it helps avoid overlapping labels. Often label data come with coordinates which are unevenly spaced. If you are forced to align your labels to a grid and summarize within cells, you can avoid bleed over from one label to another. This is especially important when you are working with data that has a high degree of spatial autocorrelation.

::: {.callout-note}
The MOSAIKS API is designed to predict outcomes at scales of 1 km² or larger. Custom solutions are possible for higher-resolution applications (see @sec-features-computing).  
:::

### Disadvantages of the MOSAIKS grid

The MOSAIKS grid is defined in degrees, therefore the area of a given cell varies with latitude. Near the equator, each cell is approximately 1 km², while at higher latitudes the area decreases as the distance between meridians (longitude lines) converges. 

### Choosing your resolution

We know from @sec-intro-api that the MOSAIKS API offers features at 0.01°, 0.1°, 1°, ADM2, ADM1, and ADM0 resolutions. In all cases, features are computed at 0.01° resolution and then aggregated to the desired resolution. If you plan on using the API to download features, you need to make sure your labels are at the same resolution as the features you plan to download.

## Ground Observations

### Resolution

Preparing label data for MOSAIKS depends on location, extent, time, and resolution. When observations have a resolution finer than 1 km², you must select or aggregate them to match the features you plan to use. MOSAIKS can incorporate labels from raster, point, polygon, or vector data. In @sec-labels-demo, we will demonstrate how to prepare labels for each input type.

![Examples of label data formats that can be easily integrated into the MOSAIKS pipeline. Label data of any spatial format that can be aggregated to at least the scale of 1km² (or larger) can be used directly in combination with MOSAIKS imagery features for downstream prediction tasks. Examples shown here are from Rolf et al. (2021) and include: forest cover, elevation, population, and nighttime lights datasets (all raster format); income data (polygon format); road length (vector format); and housing price data (point format). Source: [Rolf et al. 2021 Supplementary Figure 4](https://static-content.springer.com/esm/art%3A10.1038%2Fs41467-021-24638-z/MediaObjects/41467_2021_24638_MOESM1_ESM.pdf).](../images/rolf_et_al_2021-Fig_S4.png){#fig-label-agg}

### Administrative level data

In some cases, you will find labels for administrative regions (states, districts, etc.). In this case, it might not come with any geographic information other than a place name. You can usually find the relevant geographic information online. A good resource for this is the [Global Administrative Areas (GADM)](https://gadm.org/) database, which provides the boundaries for administrative division at various levels for completely free. 

![Global Human Development Index (HDI) data at the first sub-national level of administrative division (ADM1). Source: [Smits & Permanyer 2019](https://www.nature.com/articles/sdata201938).](../images/smits_and_permanyer_2019_HDI.png){#fig-hdi}

### Challenges of administrative data

Data can be messy. There are two main challenges with administrative data. 

1. **Name matching:** The names in your dataset might not match perfectly with the names in the GADM database or in the MOSAIKS API. Finding a way to comprehensively match these can be time consuming and difficult. 

2. **Boundary changes:** Administrative boundaries are not always static. Some regions undergo frequent changes and you need to ensure your data matches the boundaries of the features you use. 

### Sample Size

Increasing sample size (*N*) often yields higher model performance. MOSAIKS has demonstrated effectiveness across a wide range of sample *N*. The sample size depends on the spatial (and potentially temporal) resolution of your label data. For instance, if each record is aggregated at the county level, then *N* equals the number of counties. Incorporating multiple time periods can increase *N* but also requires more imagery to be featurized (see @sec-features-computing).

::: {.callout-note}
As a general rule, a minimum of 300 observations for model training is recommended, though every application is unique and may require more or fewer observations.  
:::

The original MOSAIKS publication (Rolf et al., 2021) evaluated models with sample sizes from 60,000 to 100,000. In most cases there were only modest performance declines with a few hundred observations (@fig-sample-size). In recent crop yield experiments, high performance (R² = 0.80) was achieved with around 400 observations, provided the data were cleaned and aggregated to a district level. It is important to note that the original crop yield dataset included interview records from thousands of farmers across the study country. While this data has a large sample size, it is messy. In this case, a clean dataset with a low number of observations was preferred to a large but noisy dataset. 

![Model performance for all seven tasks while varying the number of random convolutional features K and holding *N* = 64,000 (left) and while varying N and holding *K* = 8,192 (right). Shaded bands indicate the range of predictive skill across five folds. Lines indicate average accuracy across validation folds. Source: [Rolf et al. 2021 Figure 3 b](https://www.nature.com/articles/s41467-021-24638-z).](../images/rolf_et_al_2021-Fig_3-b.svg){#fig-sample-size}

### Data Types

MOSAIKS can accommodate both continuous labels (e.g., fraction of area forested) and discrete labels (e.g., presence/absence of mine). Data type informs model development, performance evaluation, and choice of metric (see @sec-model-choice). Continuous variables often use coefficient of determination (R²), while discrete variables often use Receiver Operating Characteristic Area Under the Curve (ROC AUC).

![The coefficient of determination (R²) is a measure of how well the model fits the data. It ranges from 0 to 1, where 1 indicates a perfect fit. The Receiver Operating Characteristic (ROC) curve is a graphical representation of the true positive rate (sensitivity) against the false positive rate (1-specificity). The area under the curve (AUC) ranges from 0 to 1, where 1 indicates a perfect model.](../images/R2_ROC_side_by_side_plots.png){#fig-r2-roc}

The data type may also effect how the data is cleaned and prepared. For example you may have a dataset of mining locations across a country. If you are interested in predicting the presence of mining, you may want to convert this to a binary variable where a 0 indicates no mining and a 1 indicates mining. Alternatively, you may be interested in predicting the area of mining in each location, in which case you might need to calculate the area of the mining polygons to make the variable continuous.

## Joining Data

### Merging Labels with MOSAIKS Features

To merge labels with features, you must align the geographic location of both datasets. The native resolution MOSAIKS feature files have rows for each location and columns with longitude, latitude, and features. Your label dataset must also have columns for longitude, latitude, and label values (at minimum). Alternatively, aggregated features will come with the name of the administrative region (e.g., district) and the features. You can join this with your label data by matching the district names.

### Example Data Structure

A simple example: district-level crop yield labels might look like:

| Observation | District  | Year | Crop Yield |
|-------------|-----------|------|------------|
| 1           | Chibombo  | 2019 | 1.520      |
| 2           | Kabwe     | 2019 | 1.878      |
| ...         | ...       | ...  | ...        |
| *N*         | Kitwe     | 2019 | 2.383      |

: Fictional crop yield data for districts in Zambia. {#tbl-crop-yield}

MOSAIKS features at this same resolution might look like:

| Observation | District  | Year | Feature 1 | Feature 2 | ... | Feature *K* |
|-------------|-----------|------|-----------|-----------|-----|-------------|
| 1           | Chibombo  | 2019 | 4.2       | 11.6      | ... | 12.7        |
| 2           | Kabwe     | 2019 | 2.9       | 5.3       | ... | 11.2        |
| ...         | ...       | ...  | ...       | ...       | ... | ...         |
| *N*         | Kitwe     | 2019 | 10.6      | 1.1       | ... | 2.2         |

: Fictional MOSAIKS feature data for the same districts. {#tbl-features}

After a spatial join, you end up with a merged dataset:

| Observation | District  | Year | Crop Yield | Feature 1 | Feature 2 | ... | Feature *K* |
|-------------|-----------|------|------------|-----------|-----------|-----|-------------|
| 1           | Chibombo  | 2019 | 1.520      | 4.2       | 11.6      | ... | 12.7        |
| 2           | Kabwe     | 2019 | 1.878      | 2.9       | 5.3       | ... | 11.2        |
| ...         | ...       | ...  | ...        | ...       | ...       | ... | ...         |
| *N*         | Kitwe     | 2019 | 2.383      | 10.6      | 1.1       | ... | 2.2         |

: Example of joined data with both labels and features. {#tbl-joined-data}

In the above example, our geographic location is the district and our label is the crop yield (mt/ha). We then have *K* columns containing the features and *N* observations. 

## Data Cleaning Considerations

### Coordinate Reference Systems (CRS)

The default coordinate reference system used by MOSAIKS is World Geodetic System 84 (WGS 84). The standardized code that defines WGS 84 is ["EPSG:4326"](https://epsg.io/4326). EPSG stands for the [European Petroleum Survey Group](https://en.wikipedia.org/wiki/EPSG_Geodetic_Parameter_Dataset), which maintains a database of coordinate systems and projections. The WGS 84 coordinate system is the most commonly used coordinate system for GPS data. 

![[Nine small-scale map projections.](https://www.researchgate.net/publication/273517879_User_preferences_for_world_map_projections)](https://www.researchgate.net/profile/Bojan-Savric/publication/273517879/figure/fig1/AS:347863067447297@1459948426262/The-nine-small-scale-map-projections-used-in-the-paired-comparison-test-arranged-by.png){#fig-map-projections}

If your data is not in WGS 84, you will need to reproject it to this coordinate system before joining it with MOSAIKS features.

### Label Quality

Confirm that label values are within expected ranges, deal with any outliers or missing data, and ensure units are consistent and numeric fields are properly formatted. This book does not go into a great deal of detail on cleaning messy data. This topic is covered in exhaustive detail in the book [R for Data Science](https://r4ds.had.co.nz/). 

### Temporal Alignment

If you have time series labels, you will need to compute custom features for each time period. See @sec-features-computing for more information on how to do this and @sec-model-temporal for guidance on modeling time series data with MOSAIKS features.

### Data Formats

MOSAIKS can work with several common spatial data formats:

| Data Type | Common File Formats | Description | Example | Geographic information |
|:----------|:--------------------|:------------|:--------|:-----------------------|
| Point Data | CSV, GeoJSON, Shapefile | Coordinate pairs | Places of interest | geometry |
| Line Data | GeoJSON, Shapefile | Geographic lines | Roads, rivers | geometry |
| Polygon | GeoJSON, Shapefile | Geographic areas | Buildings, fields | geometry |
| Raster | GeoTIFF, NetCDF | Gridded data | Forest cover, elevation | grid |
| Administrative | CSV | Administrative boundaries | Districts, states | place names |

: Common spatial data formats with file types, descriptions, examples, and indication of how the spatial information is stored. {#tbl-spatial-data-formats}

::: {.callout-tip}
When working with large datasets, converting to efficient data formats such as Parquet, Feather, GeoTIFF, or Zarr can reduce memory usage and improve processing speed.
:::

## Summary

The most important considerations when preparing labels for MOSAIKS are spatial resolution, sample size, data type, and data quality. Once you have prepared your labels, you can join them with MOSAIKS features to create a dataset ready for modeling.

A demonstration of how to process data for the data types in @tbl-spatial-data-formats is provided in the next chapter. This notebook will cover the process of creating a MOSAIKS grid, downloading data, and creating a label dataset at the grid level (0.01 degree) and at the second level of administrative division (ADM2).

### Labels data checklist

For optimal use with MOSAIKS, label data should be:

- [ ] **Consistently geolocated** as point, polygon, vector, or raster data
- [ ] **Aggregable to ≥ 1km²** resolution
- [ ] **Observable** in daytime satellite imagery (directly or indirectly)
- [ ] **Recent or slow-changing** if using current API features
- [ ] **Sample size N≥300** (larger samples generally perform better)
- [ ] **Cleaned and formatted** for modeling

::: {.callout-note}
# Looking forward

In the next chapter, we'll look at practical guidance for preparing label data for use with MOSAIKS including data cleaning and aggregation.
:::


<!-- ```{python}
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, roc_curve, roc_auc_score

# Data for plot A
np.random.seed(42)
observed = np.random.uniform(10, 100, 50)
predicted = observed + np.random.normal(0, 10, 50)
r2 = r2_score(observed, predicted)
line = np.linspace(min(observed.min(), predicted.min()), max(observed.max(), predicted.max()), 100)

# Data for plot B
y_true = np.random.randint(0, 2, 1000)
y_scores = np.where(
    y_true == 1,
    np.random.normal(0.7, 0.1, 1000),
    np.random.normal(0.4, 0.1, 1000)
)
y_scores = np.clip(y_scores, 0, 1)
fpr, tpr, thresholds = roc_curve(y_true, y_scores)
auc = roc_auc_score(y_true, y_scores)

# Create a figure with two subplots side by side
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Plot A: R² Demonstration
axes[0].scatter(observed, predicted, color='blue', label=f'Geographic locations $R^2 = {r2:.2f}$', alpha=0.7)
axes[0].plot(line, line, color='red', linestyle='--', label='45° Line (perfect prediction)')
axes[0].set_xlabel('Observed')
axes[0].set_ylabel('Predicted')
axes[0].set_title('A: Coefficient of Determination Demonstration')
axes[0].legend(loc='lower right')
axes[0].grid(alpha=0.5)
axes[0].axis('equal')

# Plot B: ROC Curve
axes[1].plot(fpr, tpr, color='blue', label=f'ROC Curve (AUC = {auc:.2f})')
axes[1].plot([0, 1], [0, 1], color='red', linestyle='--', label='Random Guessing')
axes[1].set_xlabel('False Positive Rate (FPR)')
axes[1].set_ylabel('True Positive Rate (TPR)')
axes[1].set_title('B: Receiver Operating Characteristic Curve')
axes[1].legend()
axes[1].grid(alpha=0.5)

# Adjust layout and show the combined figure
plt.tight_layout()

# Save the figure to the 'images' folder
output_path = '../images/R2_ROC_side_by_side_plots.png'
# os.makedirs(os.path.dirname(output_path), exist_ok=True)
plt.savefig(output_path, dpi=300)
plt.show()

``` -->